In [4]:
import csv
import pickle

import numpy as np
import h5py

In [37]:
path_root = '../'
path_container = './Container/'

In [8]:
path_train = path_root + "train.csv"
path_valid_class_que = path_root + "validation_classification_question.csv"
path_valid_class_ans = path_root + "validation_classification_answer.csv"
path_valid_compl_que = path_root + "validation_completion_question.csv"
path_valid_compl_ans = path_root + "validation_completion_answer.csv"
path_test_class_que = path_root + "test_classification_question.csv"
path_test_compl_que = path_root + "test_completion_question.csv"
path_ingredient_name = path_root + "node_ingredient.txt"

### Data 읽어서 list로 일단 저장

In [14]:
def read_train_data(path):
    """ Read train.csv and Return lists of data[int] / label[str]. """
    data = []
    labels = []
    with open(path, 'r') as f:
        for line in csv.reader(f):
            recipe = sorted(map(int, line[:-1]))  # a sorted list of recipe (integer)
            cuisine = line[-1]                    # which country? (string)
            data.append(recipe)
            labels.append(cuisine)
    return data, labels


def read_classification_data(question_path, answer_path=None):
    """ Read valid/test data for classification. Then return lists of data[int] / label[str]. """
    data = []
    labels = [] if answer_path is not None else None
    with open(question_path, 'r') as f:
        for line in csv.reader(f):
            recipe = sorted(map(int, line))  # a sorted list of recipe (integer)
            data.append(recipe)
    if answer_path is not None:
        with open(answer_path, 'r') as f:
            for line in csv.reader(f):
                cuisine = line[0]            # which country? (string)
                labels.append(cuisine)
    return data, labels


def read_completion_data(question_path, answer_path=None):
    """ Read valid/test data for completion. Then return lists of data[int] / label[str]. """
    data = []
    labels = [] if answer_path is not None else None
    with open(question_path, 'r') as f:
        for line in csv.reader(f):
            recipe = sorted(map(int, line))                # recipe without an ingredient
            data.append(recipe)
    if answer_path is not None:
        with open(answer_path, 'r') as f:
            for line in csv.reader(f):
                recipe = set(map(int, line))               # original recipe
                missing = recipe - set(data[len(labels)])  # missing ingredient in data
                labels.append(list(missing)[0])
    return data, labels


def read_ingredient_names(ingredient_path):
    ingredients_names = []
    with open(ingredient_path, 'r') as f:
        for line in csv.reader(f):
            ingredients_names.append(line[0])
    return ingredients_names

In [15]:
data_train, labels_train = read_train_data(path_train)
data_valid_class, labels_valid_class = read_classification_data(path_valid_class_que, path_valid_class_ans)
data_valid_compl, labels_valid_compl = read_completion_data(path_valid_compl_que, path_valid_compl_ans)
data_test_class, _ = read_classification_data(path_test_class_que, None)
data_test_compl, _ = read_completion_data(path_test_compl_que, None)
ingredient_names = read_ingredient_names(path_ingredient_name)  # 재료 이름 (string)
cuisine_names = sorted(set(labels_train+labels_valid_class))       # Cuisine 이름 (string), 알파벳 순

### List (```ingredient_names```, ```cuisine_names```) 를 Dictionary로 변환

In [16]:
# Dict: id (int, 0~19) -> cuisine name (str, 알파벳 순)
id_cuisine_dict = dict(zip(range(len(cuisine_names)), cuisine_names))
print(id_cuisine_dict)

{0: 'brazilian', 1: 'british', 2: 'cajun_creole', 3: 'chinese', 4: 'filipino', 5: 'french', 6: 'greek', 7: 'indian', 8: 'irish', 9: 'italian', 10: 'jamaican', 11: 'japanese', 12: 'korean', 13: 'mexican', 14: 'moroccan', 15: 'russian', 16: 'southern_us', 17: 'spanish', 18: 'thai', 19: 'vietnamese'}


In [17]:
# Dict: cuisine name (str) -> id (int)
cuisine_id_dict = {b: a for a, b in id_cuisine_dict.items()}
print(cuisine_id_dict)

{'brazilian': 0, 'british': 1, 'cajun_creole': 2, 'chinese': 3, 'filipino': 4, 'french': 5, 'greek': 6, 'indian': 7, 'irish': 8, 'italian': 9, 'jamaican': 10, 'japanese': 11, 'korean': 12, 'mexican': 13, 'moroccan': 14, 'russian': 15, 'southern_us': 16, 'spanish': 17, 'thai': 18, 'vietnamese': 19}


In [21]:
# Dict: id (int, 0~6713) -> ingredient name (str, node_ingredient.txt 기준)
id_ingredient_dict = dict(zip(range(len(ingredient_names)), ingredient_names))
print(dict(list(id_ingredient_dict.items())[:20]), '...')  # 20개만 출력

{0: 'coca-cola', 1: 'vegan butter', 2: 'sourdough rolls', 3: 'reduced sodium refried beans', 4: 'ramen noodles', 5: 'crumbled corn bread', 6: 'japanese breadcrumbs', 7: 'toasted shredded coconut', 8: 'chinese spinach', 9: "Hellmann's® Real Mayonnaise", 10: 'ducklings', 11: 'basil olive oil', 12: 'white baking bar', 13: 'rye whiskey', 14: 'mushroom broth', 15: 'meat loaf mix', 16: 'cocktail sauce', 17: 'asparagus spears', 18: 'nonfat greek yogurt', 19: 'cabernet sauvignon'} ...


In [22]:
# Dict: ingredient name (str) -> id (int, 0~6713)
ingredient_id_dict = {b: a for a, b in id_ingredient_dict.items()}
print(dict(list(ingredient_id_dict.items())[:20]), '...')  # 20개만 출력

{'coca-cola': 0, 'vegan butter': 1, 'sourdough rolls': 2, 'reduced sodium refried beans': 3, 'ramen noodles': 4, 'crumbled corn bread': 5, 'japanese breadcrumbs': 6, 'toasted shredded coconut': 7, 'chinese spinach': 8, "Hellmann's® Real Mayonnaise": 9, 'ducklings': 10, 'basil olive oil': 11, 'white baking bar': 12, 'rye whiskey': 13, 'mushroom broth': 14, 'meat loaf mix': 15, 'cocktail sauce': 16, 'asparagus spears': 17, 'nonfat greek yogurt': 18, 'cabernet sauvignon': 19} ...


### Data, Label lists를 np.ndarray로 변환

In [26]:
def data_to_binary_array(data, dim):
    """ convert data(list of lists) into a 2D binary array. (for dataset, row = recipe) """
    """ dim (int) : dimension of each row (of 'enc') that must be. """
    enc = np.zeros((len(data), dim), dtype=int) 
    for i in range(len(data)):
        recipe = data[i]
        enc[i][recipe] = 1
    return enc

num_ingredients = len(ingredient_names)
num_cuisines = len(cuisine_names)

In [33]:
# Data
bin_data_train = data_to_binary_array(data_train, num_ingredients)
bin_data_valid_class = data_to_binary_array(data_valid_class, num_ingredients)
bin_data_valid_compl = data_to_binary_array(data_valid_compl, num_ingredients)
bin_data_test_class = data_to_binary_array(data_test_class, num_ingredients)
bin_data_test_compl = data_to_binary_array(data_test_compl, num_ingredients)

for x in [bin_data_train, bin_data_valid_class, bin_data_valid_compl, bin_data_test_class, bin_data_test_compl]:
    print(x.shape)

(23547, 6714)
(7848, 6714)
(7848, 6714)
(3924, 6714)
(3924, 6714)


In [41]:
# String이던 Label을 id로 바꾸기
int_labels_train = np.array([cuisine_id_dict[label] for label in labels_train])
int_labels_valid_class = np.array([cuisine_id_dict[label] for label in labels_valid_class])
int_labels_valid_compl = np.array(labels_valid_compl)  # 원래 int형.

for x in [int_labels_train, int_labels_valid_class, int_labels_valid_compl]:
    print(x.shape)

(23547,)
(7848,)
(7848,)


In [35]:
# int형 label을 one-hot으로 바꾸기
bin_labels_train = data_to_binary_array(int_labels_train, num_cuisines)
bin_labels_valid_class = data_to_binary_array(int_labels_valid_class, num_cuisines)
bin_labels_valid_compl = data_to_binary_array(labels_valid_compl, num_ingredients)

for x in [bin_labels_train, bin_labels_valid_class, bin_labels_valid_compl]:
    print(x.shape)

(23547, 20)
(7848, 20)
(7848, 6714)


### Dictionary를 pickle로 저장

In [39]:
# %mkdir {path_container}

In [40]:
for d, name in zip([id_cuisine_dict, cuisine_id_dict, id_ingredient_dict, ingredient_id_dict],
                   ['id_cuisine_dict', 'cuisine_id_dict', 'id_ingredient_dict', 'ingredient_id_dict']):
    with open(path_container + name +'.pickle', 'wb') as fw:
        pickle.dump(d, fw)

### np.ndarray를 h5py로 저장

In [42]:
_train = [bin_data_train, int_labels_train, bin_labels_train]
_valid_class = [bin_data_valid_class, int_labels_valid_class, bin_labels_valid_class]
_valid_compl = [bin_data_valid_compl, int_labels_valid_compl, bin_labels_valid_compl]
_test_class = [bin_data_test_class, None, None]
_test_compl = [bin_data_test_compl, None, None]

In [43]:
for (bin_data, int_labels, bin_labels), name in zip([_train, _valid_class, _valid_compl, _test_class, _test_compl],
                                                    ['train', 'valid_class', 'valid_compl', 'test_class', 'test_compl']):
    with h5py.File(path_container + name, 'w') as h5f:
        h5f.create_dataset('bin_data_' + name, data=bin_data, compression="gzip")
        if 'test_' not in name:
            h5f.create_dataset('int_labels_' + name, data=int_labels, compression="gzip")
            h5f.create_dataset('bin_labels_' + name, data=bin_labels, compression="gzip")